In [1]:
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
region = 6
PCF = 553
mother_directory = f"/content/drive/MyDrive/MS thesis/[optimize] Training, Testing/RG{region}/"
existing_region_rhus = pd.read_csv(f"/content/drive/MyDrive/MS thesis/preliminary site selection/RG{region}/rg{region}_clusters.csv")
raw_rg1_clustered = pd.read_csv(mother_directory + "PCF_data.csv")
# use pcf data nalang para you dont get nans for sure?
neighbors_df = pd.read_csv(mother_directory + 'neighbours.csv')
candidate_sites = pd.read_csv(mother_directory + "candidate_sites.csv")
results = pd.read_csv(mother_directory + "results.csv")

# raw_rg1_clustered = raw_rg1_clustered.drop(columns=['Unnamed: 0'])
neighbors_df.rename(columns={'fid': 'ID'}, inplace=True)
# raw_rg1_clustered = raw_rg1_clustered.drop(columns=['HCFAI.1'])
raw_rg1_clustered.columns

Index(['flood_probability_value', 'rain intensity_value', 'drought_value',
       'Distance_to_Nearest_RHU_km', 'popden_chi', 'popden_eld', 'popden_all',
       'popden_wom', 'popden_w_1', 'popden_you', 'HCFAI', 'total_population',
       'RHU_Presence', 'ID', 'buildability_landcov', 'Road_Presence',
       'POI_Presence', 'Nearest_RHU', 'Cluster'],
      dtype='object')

## Updated

In [3]:
def HCI_calc(total_ai, total_gi, total_hi, total_ji, total_ki, total_mi, distance, road_bi, POI_ci, landCov_di, hazard1_ei, hazard2_ei, hazard3_ei, rhus_fi):
    total_vulnerable = total_gi + total_hi + total_ji + total_ki + total_mi
    total_pop = total_ai
    population_to_be_served = total_vulnerable + np.maximum(0, total_pop - total_vulnerable)
    y = np.where(population_to_be_served == 0, 0, 20000 / ((population_to_be_served) * (distance + rhus_fi)))
    mc = np.tanh(y)
    w_bi = 0.3  # roads
    w_ci = 0.2  # POIs
    w_di = 0.5  # land cov
    b = (POI_ci * w_ci) + (road_bi * w_bi) + (landCov_di * w_di)
    rain_intensity_normalized = (hazard1_ei - hazard1_ei.min()) / (hazard1_ei.max() - hazard1_ei.min())
    flood_probability_normalized = (hazard2_ei - hazard2_ei.min()) / (hazard2_ei.max() - hazard2_ei.min())
    drought_mean_normalized = (hazard3_ei - hazard3_ei.min()) / (hazard3_ei.max() - hazard3_ei.min())
    w_rain = 0.4
    w_flood = 0.3
    w_drought = 0.3
    c = (w_rain * rain_intensity_normalized) + (w_flood * flood_probability_normalized) + (w_drought * drought_mean_normalized)
    f = b - c
    f = np.tanh(f)
    hci = mc * f
    hcfai = (1 + np.tanh(hci / 2)) / 2  # Sigmoid function

    return hcfai

def overallHCFAI(region_df):
    if 'HCFAI' in region_df.columns:
        HCFAI_overall = region_df['HCFAI'].sum()
    elif 'new HCFAI' in region_df.columns:
        HCFAI_overall = region_df['new HCFAI'].sum()
    return HCFAI_overall

def selectTopSites(candidate_sites, region_df, selected_sites, n):
    if len(selected_sites) < n:  # if optimal and existing RHUs list is incomplete
        while len(selected_sites) < n:
            new_site = candidate_sites.iloc[:1].copy()
            selected_sites = pd.concat([selected_sites, new_site], ignore_index=True)
            candidate_sites = candidate_sites.drop(candidate_sites.index[0]).reset_index(drop=True)

    region_df = region_df.drop('total_population', axis=1)

    columns_to_merge = ['ID',
                        'popden_chi', 'popden_eld', 'popden_wom', 'popden_you', 'popden_w_1',
                        'popden_all', 'flood_probability_value', 'rain intensity_value',
                        'drought_value', 'buildability_landcov', 'RHU_Presence',
                        'Road_Presence', 'POI_Presence', 'Nearest_RHU',
                        'Distance_to_Nearest_RHU_km']

    merged_sites = candidate_sites.merge(region_df[columns_to_merge], on='ID', how='left')
    merged_sites['HCFAI'] = HCI_calc(merged_sites['popden_all'],
                                     merged_sites['popden_chi'],
                                     merged_sites['popden_eld'],
                                     merged_sites['popden_wom'],
                                     merged_sites['popden_you'],
                                     merged_sites['popden_w_1'],
                                     merged_sites['Distance_to_Nearest_RHU_km'],
                                     merged_sites['Road_Presence'],
                                     merged_sites['POI_Presence'],
                                     merged_sites['buildability_landcov'],
                                     merged_sites['rain intensity_value'],
                                     merged_sites['flood_probability_value'],
                                     merged_sites['drought_value'],
                                     merged_sites['RHU_Presence'],
                                     )

    missing_cols = [col for col in region_df.columns if col != 'ID']
    for col in missing_cols:
        if 'ID' not in region_df.columns:
            region_df.set_index('ID', inplace=True)
        merged_sites[col] = merged_sites[col].fillna(region_df[col])

    top_sites = merged_sites.sort_values(by='HCFAI', ascending=False).head(n)
    return top_sites

def removeAdjacentSites(region_df, candidate_sites, selected_sites, neighbors_df, n):
    idx_with_RHU = region_df[region_df['RHU_Presence'] == 1]
    adjacent_sites = set()
    with_RHU_indices = selected_sites['ID'].tolist()
    for site_id in selected_sites['ID']:
        if site_id in neighbors_df['ID'].values:
            neighbors = neighbors_df.loc[neighbors_df['ID'] == site_id, 'neighbours'].iloc[0]
            adjacent_sites.update(neighbors.split(','))
    adjacent_sites.update(idx_with_RHU['ID'])
    adjacent_sites.update(selected_sites['ID'])
    adjacent_sites = [int(site) for site in adjacent_sites]
    candidate_sites = candidate_sites[~candidate_sites['ID'].isin(adjacent_sites)].reset_index(drop=True)

    print(f"{len(candidate_sites)} Candidate sites left")
    print(f"This is what we'll put on QGIS: {selected_sites['ID'].tolist()}")

    if len(candidate_sites) > n:
      for i in with_RHU_indices:
          print(f"Site {i} has the following sites to choose from: {candidate_sites['ID'].tolist()}")
          if i in adjacent_sites:
              print(f"Site {i} was highkey sus for not saying they have neighbors ...")
              with_RHU_indices.remove(i)
              selected_sites = selected_sites.drop(selected_sites[selected_sites['ID'] == i].index).reset_index(drop=True)
    elif len(candidate_sites) <= n:
      return candidate_sites, selected_sites, idx_with_RHU

    selected_sites['HCFAI'] = HCI_calc(selected_sites['popden_all'], selected_sites['popden_chi'],
                                       selected_sites['popden_eld'], selected_sites['popden_wom'],
                                       selected_sites['popden_you'], selected_sites['popden_w_1'],
                                       selected_sites['Distance_to_Nearest_RHU_km'],
                                       selected_sites['Road_Presence'], selected_sites['POI_Presence'],
                                       selected_sites['buildability_landcov'], selected_sites['rain intensity_value'],
                                       selected_sites['flood_probability_value'], selected_sites['drought_value'],
                                       selected_sites['RHU_Presence'])
    return candidate_sites, selected_sites, idx_with_RHU

def optimize(region_df, candidate_sites, neighbors_df, num_facilities):
    original_HCFAI = overallHCFAI(region_df)
    print("Original HCFAI:", original_HCFAI)
    selected_sites = pd.DataFrame(columns=region_df.columns)
    top_sites = selectTopSites(candidate_sites, region_df, selected_sites, num_facilities - len(selected_sites))

    while len(selected_sites) < num_facilities:
        candidate_sites, top_sites, idx_with_RHU = removeAdjacentSites(region_df, candidate_sites, top_sites, neighbors_df, num_facilities)
        if len(candidate_sites) <= num_facilities:
          top_sites = candidate_sites
        elif len(candidate_sites) > num_facilities:
          top_sites = selectTopSites(candidate_sites, region_df, selected_sites, num_facilities - len(selected_sites))
          selected_sites = top_sites

        selected_sites = pd.concat([selected_sites, top_sites]).reset_index(drop=True)
        remaining_sites = region_df[~region_df['ID'].isin(selected_sites['ID'])]
        remaining_HCFAI = overallHCFAI(remaining_sites)
        selected_sites_HCFAI = overallHCFAI(selected_sites)
        updated_HCFAI = remaining_HCFAI + selected_sites_HCFAI
        print("Updated HCFAI:", updated_HCFAI)
        print(f"{len(selected_sites)} selected sites")

        if len(selected_sites) == num_facilities:
            print("Accept!!!!!!!!!!!!!!")
            return selected_sites, idx_with_RHU, original_HCFAI, updated_HCFAI

        elif len(candidate_sites) <= num_facilities:
            return selected_sites, idx_with_RHU, original_HCFAI, updated_HCFAI

        else:
            print("Reject!!!!!!!!!!!!!!")
            print("Selected sites:", len(selected_sites))
            top_sites = selectTopSites(candidate_sites, region_df, selected_sites, num_facilities - len(selected_sites))

    return selected_sites, idx_with_RHU, original_HCFAI, updated_HCFAI

# Generate

In [4]:
print(f"Candidate sites: {len(candidate_sites)}")
print(f"Existing RHU sites: {existing_region_rhus[existing_region_rhus['RHU_Presence'] == 1]['ID'].tolist()}")
print(list(candidate_sites['ID']))
print(list(existing_region_rhus[existing_region_rhus['RHU_Presence'] == 1]['ID']))

Candidate sites: 1999
Existing RHU sites: [3191699, 3195843, 3199993, 3222135, 3226281, 3244251, 3258091, 3258144, 3271970, 3276099, 3281579, 3282963, 3283013, 3334079, 3340987]
[3140561, 3141941, 3141943, 3143323, 3143324, 3144704, 3146087, 3146088, 3147468, 3148848, 3148849, 3148852, 3150230, 3150236, 3151612, 3151613, 3151616, 3151617, 3152988, 3152994, 3152997, 3153001, 3154372, 3154377, 3154378, 3154380, 3154381, 3154382, 3154383, 3155754, 3155758, 3155759, 3155760, 3155762, 3155763, 3155764, 3155765, 3157137, 3157138, 3157140, 3157142, 3157143, 3157145, 3158519, 3158521, 3158526, 3158527, 3159906, 3159907, 3159909, 3159913, 3161290, 3161291, 3161292, 3161293, 3161294, 3162672, 3162673, 3162674, 3162675, 3162677, 3162678, 3164053, 3164054, 3164055, 3164056, 3164069, 3165439, 3165440, 3165441, 3165450, 3165451, 3165452, 3165455, 3165456, 3166810, 3166821, 3166822, 3166823, 3166824, 3166825, 3166831, 3166832, 3166833, 3168192, 3168193, 3168204, 3168205, 3168206, 3168207, 3168208, 31

In [6]:
# PCF = 363
# selected_facilities, updated_HCFAI, og_HCFAI = optimize(raw_rg1_clustered, PCF) # 81 for dropping non buildable; 363 for validation
selected_facilities, hex_with_RHU, og_HCFAI, updated_HCFAI = optimize(raw_rg1_clustered, candidate_sites, neighbors_df, PCF)

print(f"Optimal sites: {len(selected_facilities)}")
print(selected_facilities)

Output hidden; open in https://colab.research.google.com to view.

In [10]:
os = list(selected_facilities['ID'])
cs = list(candidate_sites['ID'])
es = list(hex_with_RHU['ID'])

# hex_with_RHU = list(hex_with_RHU['ID'])
print(f"{cs}")
print(f"{list(set(os))}")
print(f"{es}")

[3140561, 3141941, 3141943, 3143323, 3143324, 3144704, 3146087, 3146088, 3147468, 3148848, 3148849, 3148852, 3150230, 3150236, 3151612, 3151613, 3151616, 3151617, 3152988, 3152994, 3152997, 3153001, 3154372, 3154377, 3154378, 3154380, 3154381, 3154382, 3154383, 3155754, 3155758, 3155759, 3155760, 3155762, 3155763, 3155764, 3155765, 3157137, 3157138, 3157140, 3157142, 3157143, 3157145, 3158519, 3158521, 3158526, 3158527, 3159906, 3159907, 3159909, 3159913, 3161290, 3161291, 3161292, 3161293, 3161294, 3162672, 3162673, 3162674, 3162675, 3162677, 3162678, 3164053, 3164054, 3164055, 3164056, 3164069, 3165439, 3165440, 3165441, 3165450, 3165451, 3165452, 3165455, 3165456, 3166810, 3166821, 3166822, 3166823, 3166824, 3166825, 3166831, 3166832, 3166833, 3168192, 3168193, 3168204, 3168205, 3168206, 3168207, 3168208, 3168214, 3168215, 3168216, 3169571, 3169572, 3169573, 3169574, 3169586, 3169587, 3169588, 3169589, 3169590, 3169596, 3169597, 3169598, 3170954, 3170955, 3170956, 3170957, 3170964, 

# Validation

In [11]:
# Python program to get average of a list
def getAvgHCFAI(lst):
    return sum(lst) / len(lst)

def getSumHCFAI(lst):
  return sum(lst)

def getRandom4(candidate_sites):
  candidate_sites = candidate_sites.merge(results, on=['ID']) # arrange in descending order
  candidate_sites = candidate_sites.rename(columns={"HCFAI": "original HCFAI"}) # renamed HCFAI into "original HCFAI"
  random4 = candidate_sites.sample(n=PCF)
  return random4

def randomize(sablayan_df):
  # use clustered to get top 4
  # use raw for the overall
  raw_sablayan_clustered_df = raw_rg1_clustered
  # raw_sablayan_clustered_df.rename(columns = {'HCFAI_y':'HCFAI'}, inplace = True)

  HCFAI_overall = overallHCFAI(raw_sablayan_clustered_df)
  print("=========== Overall HCFAI (before) ============ ", HCFAI_overall)

  # get the list of ids of predicted locations
  random4 = getRandom4(sablayan_df)
  random4 = recomputeHCFAI(random4)
  sablayan_df = updateOverallHCFAIValue(raw_sablayan_clustered_df, random4)
  HCFAI_overall = overallHCFAI(sablayan_df)
  rdm4_list = random4['ID'].tolist()
  # print("This is random4: ", random4['id'].tolist())
  # print("This is random4's HCFAI: ", random4['updated HCFAI'].tolist())
  print("=========== Overall HCFAI (after) ============ ", HCFAI_overall)

  return HCFAI_overall, rdm4_list

rdm_hcfai = list()
rdm_points = list()
repeat = 0
while repeat < 10:
  rdm_ovHCFAI, rdm4 = randomize(raw_rg1_clustered)
  rdm_hcfai.append(rdm_ovHCFAI)
  rdm_points.append(rdm4)
  repeat+=1

print(rdm_hcfai)
rdm_HCFAI = getAvgHCFAI(rdm_hcfai)
print("New avg hcfai [[RANDOM]]: ", rdm_HCFAI)


print("===========================================")
print("=========== ORIGINAL RESULTS ===========")
print(f"ORIGINAL HCFAI: {og_HCFAI}")
print("=========== RANDOM RESULTS ===========")
print("RANDOMIZED (AVG) HCFAI: ", rdm_HCFAI)
print("============ OPTIMIZED RESULTS ==========")
# print("Optimal locations: ", optimal_locations)
print("OPTIMAL HCFAI: ", updated_HCFAI)

=========== Overall HCFAI (before) ============  3129.548999082882


NameError: name 'recomputeHCFAI' is not defined

In [12]:
hcfai_compare_df =  pd.DataFrame(
    {' ': ['SUM'],
    'Old HCFAI': og_HCFAI,
     'Randomized HCFAI': rdm_HCFAI,
     'Algorithm HCFAI': updated_HCFAI
    })

import matplotlib.pyplot as plt
# Preparing the data for plotting
# Since we have only one row with label 'SUM', we'll use it as index
hcfai_compare_df.set_index(' ', inplace=True)
hcfai_compare_df = hcfai_compare_df.transpose()  # Transpose for easier plotting

# Plotting the bar chart
hcfai_compare_df.plot(kind='bar', legend=False)
plt.title(f'HCFAI Comparison: Region{region-1}')
plt.ylabel('HCFAI Value')
plt.ylim(min([og_HCFAI, rdm_HCFAI, updated_HCFAI])-5, max([og_HCFAI, rdm_HCFAI, updated_HCFAI])+2)
plt.xlabel('Method')
plt.xticks(rotation=45)
plt.show()

NameError: name 'rdm_HCFAI' is not defined